# 1. 메모리
**LangGraph 기반 메모리 권장**
- LangChain 1.0에서는 복잡한 상태 관리가 필요한 경우 LangGraph 기반 메모리(`InMemorySaver`, `SqliteSaver` 등)를 권장합니다.

**메모리 역할**

1. 대화 기록 저장: 이전 메시지들을 저장하고 관리

2. 컨텍스트 유지: 대화의 흐름과 맥락 보존

3. 개인화: 사용자별 대화 세션 관리

4. 효율성: 토큰 사용량 최적화를 위한 메모리 관리

**메모리 유형**

**1. 단기 메모리 (Short-term Memory)**
- 현재 대화 세션 동안만 유지되는 메모리

| 유형 | 설명 | 토큰 사용 | 적합한 사용 사례 |
|---|---|---|---|
| 전체 히스토리 | 모든 메시지를 그대로 저장 | 높음 | 짧은 대화, 정확한 맥락 필요 |
| 윈도우 메모리 | 최근 N개 메시지만 유지 | 중간 | 일반적인 챗봇 |
| 요약 메모리 | 이전 대화를 요약하여 저장 | 낮음 | 긴 대화, 비용 최적화 |
| 토큰 버퍼 | 토큰 수 기준으로 자동 트리밍 | 제어 가능 | 토큰 예산 관리 |

**2. 장기 메모리 (Long-term Memory)**
- 여러 세션에 걸쳐 유지되는 영구적 메모리

| 유형 | 설명 | 저장소 | 적합한 사용 사례 |
|---|---|---|---|
| 사용자 프로필 | 사용자 선호도, 속성 저장 | DB, Store | 개인화된 응답 |
| 대화 요약 | 과거 세션 요약 저장 | DB, Store | 세션 간 맥락 유지 |
| 지식 추출 | 대화에서 핵심 정보 추출 | Vector DB | 장기 학습 에이전트 |

### LangGraph 기반 메모리

In [3]:
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents import create_agent

# 체크포인터로 상태 저장
checkpointer = InMemorySaver()

agent_with_memory = create_agent(
    model="gpt-4o-mini",
    tools=[],
    checkpointer=checkpointer  # 자동 상태 저장
)

# thread_id로 대화 구분
config = {"configurable": {"thread_id": "conversation_1"}}

# 첫 번째 대화
response1 = agent_with_memory.invoke(
    {"messages":[{"role": "user", "content": "안녕하세요"}]},
    config=config
)

print("AI:", response1['messages'][-1].content)
print("-" * 50)

# 두 번째 대화 (이전 대화 기억)
response2 = agent_with_memory.invoke(
    {"messages":[{"role": "user", "content": "내 이름은 김철수입니다."}]}, 
    config=config
)
print("AI:", response2['messages'][-1].content)
print("-" * 50)

# 세 번째 대화 (이름 기억 확인)
response3 = agent_with_memory.invoke(
    {"messages":[{"role": "user", "content": "내 이름이 뭐였죠?"}]}, 
    config=config
)
print("AI:", response3['messages'][-1].content)

AI: 안녕하세요! 무엇을 도와드릴까요?
--------------------------------------------------
AI: 김철수님, 반갑습니다! 어떻게 도와드릴까요?
--------------------------------------------------
AI: 김철수님이시라고 말씀하셨습니다. 더 궁금한 점이 있으면 언제든지 말씀해 주세요!
